In [11]:
import pandas as pd
import numpy as np
import re
import json

In [12]:
COL_DICT = {'jahr': int,
 'kennnummer': str,
 'betriebsname': str,
 'betriebsname_2': str,
 'plz': str,
 'ort': str,
 'strasse': str,
 'hausnr': str,
 'bundesland': str,
 'flusseinzugsgebiet': str,
 'geo_lat_wgs84': np.float64,
 'geo_long_wgs84': np.float64,
 'taet_nr': str,
 'taetigkeit': str,
 'activity': str,
 'haupttaetigkeit': str,
 'branche': str,
 'sector': str,
 'nace_id': int,
 'nace_wirtschaftszweig': str,
 'nace_sector': str,
 'stoffgruppe': str,
 'substances_group': str,
 'schadstoff': str,
 'pollutant': str,
 'umweltkompartiment': str,
 'releases_to': str,
 'jahresfracht_freisetzung': float,
 'versehentliche_freisetzung': float,
 'schadstoff_schwellenwert': float,
 'einheit': str,
 'unit': str,
 'bestimmungsmethode': str,
 'determination_method': str,
 'schutzgrund_fracht': str,
 'confidential_reason_release': str,
 'schutzgrund_betrieb': str,
 'confidential_reason_facility': str}

DROP_LIST = ['flusseinzugsgebiet', 'schutzgrund_fracht', 'confidential_reason_release', 'schutzgrund_betrieb', 'confidential_reason_facility', 'versehentliche_freisetzung', 'bestimmungsmethode', 'determination_method', 'einheit', 'umweltkompartiment', 'haupttaetigkeit', 'taetigkeit', 'nace_wirtschaftszweig', 'schadstoff', 'stoffgruppe', 'branche', 'schadstoff_schwellenwert', 'blockid', 'geo_lat_wgs84', 'geo_long_wgs84']

In [13]:
#raw0 = pd.read_csv("../data/2020-05-19_PRTR-Deutschland_Freisetzungen.csv", encoding="iso-8859-1", sep=";", dtype=COL_DICT)

In [14]:
raw0 = pd.read_excel("../data/2022-12-15_PRTR-Deutschland_Freisetzungen - Excel.xlsx", dtype=COL_DICT)

In [15]:
raw1 = raw0.rename(columns={"jahr": "year", "betriebsname_2": "plantname", "betriebsname": "company", "kennnummer": "plantid", "ort": "place", "strasse": "street", "bundesland": "federalstate", "jahresfracht_freisetzung": "amount"})

In [16]:
raw1['plantid'] = raw1['plantid'].apply(lambda x: str(x).replace('/', '_'))

In [17]:
#raw0 = pd.read_csv("../data/2019-11-25_PRTR-Deutschland_Freisetzungen.csv", encoding="iso-8859-1", error_bad_lines=False, engine="c", sep=";", low_memory=False)

In [18]:
#raw0

In [19]:
bpm = pd.read_csv("../basic/block_plant_mapper.csv")
bpm['plantid'] = bpm['plantid'].apply(lambda x: str(x).replace('/', '_'))
bpm.drop_duplicates(["plantid"], inplace=True)

In [20]:
bpm

,plantid,blockid
0,06-05-100-0248923,BNA1401a
2,14-80-00009560000,BNA0115
4,06-05-700-0105516,BNA0793
5,06-08-2948214,BNA0646b
6,06-08-2797933,BNA0518b
...,...,...
574,06-04-11_2001178_8_0,BNA0140
575,06-04-11_2001178_8_2,BNA0141
582,661-115,BNA1334
583,06-04-11_2039669_2_0,BNA0147


In [21]:
#raw1.loc[raw1.plantid == "03-01-01012110180"]

In [22]:
raw2 = pd.merge(raw1, bpm, how="inner", on="plantid")

In [23]:
raw2.year = raw2.year.astype(int)

In [24]:
raw3 = raw2.drop(DROP_LIST, axis=1)

In [25]:
#raw3.loc[raw3.plantid == "03-01-01012110180"]

In [26]:
p = "PCDD + PCDF (dioxins + furans) (as Teq)"

In [27]:
q = "Trichlorobenzenes (TCBs) (all isomers)"
z = "Total organic carbon (TOC) (as total C or COD/3)"

In [28]:
re.findall("([A-Z]{2,}[A-Za-z]*?)\)",q)[-1]

'TCBs'

In [29]:
t = "Kraftwerk Voerde OHG der STEAG GmbH und RWE"

In [30]:
match_list = ["Vattenfall", "RWE", "Uniper", "LEAG", "EnBW", "Steag"]

In [31]:
raw4 = raw3.sort_values(by="amount", ascending=False)

In [32]:
raw3.shape

(12957, 19)

In [33]:
raw4.shape

(12957, 19)

In [34]:
def fix_company(company):
    for name in match_list:
        if name in company:
            return name
        else:
            return company

In [35]:
def fix_pollutant(pol):
    match = re.findall("([A-Z]{1,}[0-9]*[A-Za-z]+[0-9]*?)\)",pol)
    if match:
        return match[-1]
    else:
        return pol

In [36]:
fix_pollutant(q)

'TCBs'

In [37]:
unit_2_dict = {9: "Mio. t", 6: "Tsd. t", 3: "t", 1: "kg", -3: "g", -6: "mg"}
amount_dict = {"CO2": 9, "CO": 6, "NO2": 6, "Cl": 6, "SO2": 6, "Total nitrogen": 6, "PM10": 6, "HCl": 3, "HF": 3, "N2O": 6, "Benzene": 3, "Pb": 3, "Zn": 3, "Benzene": 3, "Fluorides (as total F)": 3, "NMVOC": 3, "NH3": 3, "TOC": 3, "CH4": 3, "AOX": 3, "Total phosphorus": 3, "HCFCs": 3, "HFCs": 3, "DCE": 3, "Phenols (as total C)": 3, "Cu": 3, "PER": 3, "Cr": 3, "Trichloroethylene": 3, "Trichloromethane": 3, "TCM": 3, "CN": 3, "Ni": 3, "DCM": 3, "HCN": 3, "As": 3, "Hg": 3}
neg_dict = {"Teq": 6}

In [38]:
raw4['group'] = raw4['company'].apply(lambda x: fix_company(x))
raw4['pollutant'] = raw4['pollutant'].apply(lambda x: fix_pollutant(x))

In [39]:
raw5 = raw4.copy()

In [40]:
#raw4

In [41]:
raw5['potency'] = raw5['pollutant'].apply(lambda x: amount_dict.get(x, 1))

In [42]:
raw5['neg_potency'] = raw5['pollutant'].apply(lambda x: neg_dict.get(x, 0))

In [43]:
raw5['unit_2'] = raw5['potency'].apply(lambda x: unit_2_dict[x])

In [44]:
raw5['amount_2'] = raw5['amount'] / 10**(raw5['potency'])
raw5['amount_2'] = raw5['amount_2'] * 10**(raw5['neg_potency'])

In [45]:
raw5.loc[raw5['pollutant'] == "Teq"] = raw5.loc[raw5['pollutant'] == "Teq"].replace("kg", "mg")

In [46]:
#raw5

In [47]:
DROP_LIST = ['plz', 'company','plantname', 'group', 'federalstate',  'place', 'street', 'hausnr', 'taet_nr', 'nace_id', 'substances_group', 'unit', 'activity', 'sector', 'nace_sector', 'neg_potency']

In [48]:
raw6 = raw5.drop(DROP_LIST, axis=1)

In [49]:
raw6['pollutant2'] = raw6['pollutant'] + ' [' + raw6['unit_2'] + ']'

In [50]:
raw6

,year,plantid,pollutant,releases_to,amount,potency,unit_2,amount_2,pollutant2
2689,2013,06-05-100-0248923,CO2,Air,3.330000e+10,9,Mio. t,33.3,CO2 [Mio. t]
2692,2014,06-05-100-0248923,CO2,Air,3.240000e+10,9,Mio. t,32.4,CO2 [Mio. t]
2740,2018,06-05-100-0248923,CO2,Air,3.220000e+10,9,Mio. t,32.2,CO2 [Mio. t]
2706,2015,06-05-100-0248923,CO2,Air,3.210000e+10,9,Mio. t,32.1,CO2 [Mio. t]
4120,2007,06-05-300-0326774,CO2,Air,3.130000e+10,9,Mio. t,31.3,CO2 [Mio. t]
...,...,...,...,...,...,...,...,...,...
589,2015,03-03-03030273580,Teq,Water,1.310000e-04,1,mg,13.1,Teq [mg]
2974,2012,06-05-100-0431554,Teq,Air,1.300000e-04,1,mg,13.0,Teq [mg]
10641,2015,12-40710010000,Teq,Air,1.300000e-04,1,mg,13.0,Teq [mg]
4573,2009,06-05-300-0923949,Teq,Air,1.150000e-04,1,mg,11.5,Teq [mg]


In [51]:
m1 = raw6.duplicated(['year', 'pollutant', 'plantid', 'releases_to'])
m2 = raw6.duplicated(['year', 'pollutant', 'plantid'])
m3 = raw6.duplicated(['year', 'pollutant', 'plantid', 'releases_to'])

In [52]:
raw7 = raw6.sort_values(['plantid', 'year', 'potency'], ascending=[True, False, False])

In [53]:
raw7.loc[raw7.releases_to == "Water"]

,year,plantid,pollutant,releases_to,amount,potency,unit_2,amount_2,pollutant2
424,2021,03-01-01241117210,Zn,Water,224.0,3,t,0.2240,Zn [t]
421,2021,03-01-01241117210,Ni,Water,162.0,3,t,0.1620,Ni [t]
411,2020,03-01-01241117210,Zn,Water,148.0,3,t,0.1480,Zn [t]
410,2020,03-01-01241117210,Ni,Water,106.0,3,t,0.1060,Ni [t]
401,2019,03-01-01241117210,Zn,Water,147.0,3,t,0.1470,Zn [t]
...,...,...,...,...,...,...,...,...,...
12426,2009,18042,Cl,Water,78800000.0,6,Tsd. t,78.8000,Cl [Tsd. t]
12425,2009,18042,Zn,Water,140.0,3,t,0.1400,Zn [t]
12419,2008,18042,Cl,Water,96700000.0,6,Tsd. t,96.7000,Cl [Tsd. t]
12416,2008,18042,Zn,Water,272.0,3,t,0.2720,Zn [t]


In [54]:
raw7.loc[raw7.plantid == "14-70-46630660001"].sort_values("potency", ascending=False)

,year,plantid,pollutant,releases_to,amount,potency,unit_2,amount_2,pollutant2
11804,2021,14-70-46630660001,CO2,Air,1.560000e+10,9,Mio. t,15.60,CO2 [Mio. t]
11780,2020,14-70-46630660001,CO2,Air,1.540000e+10,9,Mio. t,15.40,CO2 [Mio. t]
11656,2010,14-70-46630660001,CO2,Air,1.510000e+10,9,Mio. t,15.10,CO2 [Mio. t]
11732,2016,14-70-46630660001,CO2,Air,1.860000e+10,9,Mio. t,18.60,CO2 [Mio. t]
11766,2018,14-70-46630660001,CO2,Air,1.910000e+10,9,Mio. t,19.10,CO2 [Mio. t]
...,...,...,...,...,...,...,...,...,...
11731,2016,14-70-46630660001,Cd,Air,4.890000e+01,1,kg,4.89,Cd [kg]
11771,2019,14-70-46630660001,Cd,Air,1.400000e+01,1,kg,1.40,Cd [kg]
11730,2015,14-70-46630660001,Cd,Air,3.800000e+01,1,kg,3.80,Cd [kg]
11790,2020,14-70-46630660001,Cd,Air,2.100000e+01,1,kg,2.10,Cd [kg]


In [55]:
adds = pd.read_csv("additionals.csv") # manually add boxberg

FileNotFoundError: [Errno 2] No such file or directory: 'additionals.csv'

In [ ]:
adds

In [56]:
raw8 = raw7.append(adds)

NameError: name 'adds' is not defined

In [57]:
raw8 = raw6

In [58]:
raw8

,year,plantid,pollutant,releases_to,amount,potency,unit_2,amount_2,pollutant2
2689,2013,06-05-100-0248923,CO2,Air,3.330000e+10,9,Mio. t,33.3,CO2 [Mio. t]
2692,2014,06-05-100-0248923,CO2,Air,3.240000e+10,9,Mio. t,32.4,CO2 [Mio. t]
2740,2018,06-05-100-0248923,CO2,Air,3.220000e+10,9,Mio. t,32.2,CO2 [Mio. t]
2706,2015,06-05-100-0248923,CO2,Air,3.210000e+10,9,Mio. t,32.1,CO2 [Mio. t]
4120,2007,06-05-300-0326774,CO2,Air,3.130000e+10,9,Mio. t,31.3,CO2 [Mio. t]
...,...,...,...,...,...,...,...,...,...
589,2015,03-03-03030273580,Teq,Water,1.310000e-04,1,mg,13.1,Teq [mg]
2974,2012,06-05-100-0431554,Teq,Air,1.300000e-04,1,mg,13.0,Teq [mg]
10641,2015,12-40710010000,Teq,Air,1.300000e-04,1,mg,13.0,Teq [mg]
4573,2009,06-05-300-0923949,Teq,Air,1.150000e-04,1,mg,11.5,Teq [mg]


In [59]:
#raw8.sort_values("amount", ascending=False)

In [60]:
#pl2.loc[pl2.blockid == 'BNA1056', 'initialop'] = 2006

In [61]:
raw8.to_csv("pollutants_pg.csv", index=True, header=False)
raw8.to_csv("pollutants.csv", index=False)
raw8.to_csv("pollutants_nh.csv", index=False, header=False)

In [62]:
bpm.loc[bpm.plantid == "06-05-500-0342713"]

,plantid,blockid
132,06-05-500-0342713,BNA0189


In [111]:
#raw2.loc[raw2.plantid == "06-04-11_2039669_2_0"]

In [112]:
#raw6[m2].groupby("pollutant").max()

In [113]:
#raw6[m2].groupby("pollutant").min()

In [67]:
#raw5.loc[raw5["pollutant"] == "Teq"].sort_values("amount", ascending=False)

In [68]:
#raw4.loc[raw4['year'] == 2017].loc[raw4['plantid'] == '06-05-100-0248923'].sort_values("amount", ascending=False)

In [69]:
#raw4.loc[raw4['year'] == 2017].loc[raw4['plantid'] == '12-40710010000'].sort_values("amount", ascending=False) #12-40710010000 #06-05-300-0923949

In [70]:
#raw4.loc[raw4['year'] == 2009].loc[raw4['plantid'] == '06-05-100-0431554'].sort_values("amount", ascending=False)

In [71]:
#raw4.groupby("pollutant").sum()

In [72]:
#raw3.groupby("pollutant").sum()

In [73]:
#raw4

In [66]:
raw7.groupby('releases_to').min()

,year,plantid,pollutant,amount,potency,unit_2,amount_2,pollutant2
releases_to,,,,,,,,
Air,2007,01-50-01000004838,As,0.000110,1,Mio. t,0.010,As [t]
Land,2008,06-90031210632,AOX,15.000000,3,Tsd. t,0.015,AOX [t]
Water,2007,03-01-01241117210,AOX,0.000131,1,Tsd. t,0.001,AOX [t]


In [74]:
raw7.groupby('releases_to').min()

,year,plantid,pollutant,amount,potency,unit_2,amount_2,pollutant2
releases_to,,,,,,,,
Air,2007,01-50-01000004838,As,0.000115,1,Mio. t,0.01000,As [t]
Land,2008,06-90031210632,AOX,15.000000,3,Tsd. t,0.01500,AOX [t]
Water,2007,03-01-01241117210,AOX,0.000131,1,Tsd. t,0.00102,AOX [t]


In [75]:
raw7

,year,plantid,pollutant,releases_to,amount,potency,unit_2,amount_2,pollutant2
44,2018,01-50-01000004838,CO2,Air,602000000.0,9,Mio. t,0.602000,CO2 [Mio. t]
46,2018,01-50-01000004838,SO2,Air,208000.0,6,Tsd. t,0.208000,SO2 [Tsd. t]
45,2018,01-50-01000004838,NO2,Air,196000.0,6,Tsd. t,0.196000,NO2 [Tsd. t]
47,2018,01-50-01000004838,HCl,Air,46100.0,3,t,46.100000,HCl [t]
40,2017,01-50-01000004838,CO2,Air,597000000.0,9,Mio. t,0.597000,CO2 [Mio. t]
...,...,...,...,...,...,...,...,...,...
10194,2008,18042,Zn,Water,272.0,3,t,0.272000,Zn [t]
10195,2008,18042,Pb,Water,32.5,3,t,0.032500,Pb [t]
10191,2007,18042,CO2,Air,209038000.0,9,Mio. t,0.209038,CO2 [Mio. t]
10190,2007,18042,Cl,Water,77671600.0,6,Tsd. t,77.671600,Cl [Tsd. t]


In [63]:
'''
"text/plain": [
-       "   year            plantid pollutant releases_to       amount  potency  \\\n",
-       "0  2018  14-70-46630660001       CO2         Air  19100000000        9   \n",
-       "\n",
-       "   unit_2  amount_2    pollutant2  \n",
-       "0  Mio. t      19.1  CO2 [Mio. t]  "
-      ]
-     },

'''

'\n"text/plain": [\n-       "   year            plantid pollutant releases_to       amount  potency  \\\n",\n-       "0  2018  14-70-46630660001       CO2         Air  19100000000        9   \n",\n-       "\n",\n-       "   unit_2  amount_2    pollutant2  \n",\n-       "0  Mio. t      19.1  CO2 [Mio. t]  "\n-      ]\n-     },\n\n'